In [1]:

from typing import Sized
import warnings
import os
import torch
import torch.nn as nn
import json
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers import BertConfig, BertTokenizer
from transformers import  BertForMaskedLM
from tqdm import tqdm 
from torch.utils.data import TensorDataset, DataLoader, random_split 
from transformers import BertForSequenceClassification, AdamW 
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, RobertaForSequenceClassification
from transformers import DebertaTokenizer, DebertaModel, DebertaConfig, DebertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup 

import copy
import numpy as np 
from transformers import BertTokenizer, BertModel
import torch
import pickle
import nltk
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
supported_pos_tags = [#nltk词性标注中的一些类别
    #'CC',  # coordinating conjunction, like "and but neither versus whether yet so"  #并列连词
    'JJ',  # Adjective, like "second ill-mannered" #组合式形容词
    'JJR',  # Adjective, comparative, like "colder" #形容词比较级
    'JJS',  # Adjective, superlative, like "cheapest" 形容词最高级
    'NN',  # Noun, singular or mass #名词 单数
    'NNS',  # Noun, plural #名词复数
    'NNP',  # Proper noun, singular #专有名词单数
    'NNPS',  # Proper noun, plural  #专有名词复数
    'RB',  # Adverb#副词
    'RBR',  # Adverb, comparative, like "lower heavier"
    'RBS',  # Adverb, superlative, like "best biggest"
    'VB',  # Verb, base form
    'VBD',  # Verb, past tense 动词过去式
    'VBG',  # Verb, gerund or present participle 动名词/现在分词
    'VBN',  # Verb, past participle #过去分词
    'VBP',  # Verb, non-3rd person singular present #非第三人称单数
    'VBZ',  # Verb, 3rd person singular present #第三人称单数
]

In [3]:
def  get_pos_vocabulary():
    file_name = '/data/zhanghData/AttentionDefense/data/words_pos.dict'
    with open(file_name, 'rb') as f:
            pos_vocabulary = pickle.load(f)
    return pos_vocabulary

In [4]:
pos_vocabulary = get_pos_vocabulary()
print(len(pos_vocabulary))

28301


In [5]:
stop_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost', 
    'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anyhow', 
    'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as', 'at', 'back', 'been', 'before',
     'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot',
      'could', 'couldn', "couldn't", 'd', 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 
      'either', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 
      'first', 'for', 'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence', 
      'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 
      'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'latter', 
      'latterly', 'least', 'll', 'may', 'me', 'meanwhile', 'mightn', "mightn't", 'mine', 'more', 'moreover', 'most', 'mostly', 
      'must', 'mustn', "mustn't", 'my', 'myself', 'namely', 'needn', "needn't", 'neither', 'never', 'nevertheless', 'next', 
      'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 'off', 'on', 'once', 'one', 'only', 
      'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'per', 'please', 's', 'same', 
      'shan', "shan't", 'she', "she's", "should've", 'shouldn', "shouldn't", 'somehow', 'something', 'sometime', 'somewhere', 
      'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 
      'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'this', 'those', 'through', 'throughout', 
      'thru', 'thus', 'to', 'too', 'toward', 'towards', 'under', 'unless', 'until', 'up', 'upon', 'used', 've', 'was', 'wasn', 
      "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 
      'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 
      'whom', 'whose', 'why', 'with', 'within', 'without', 'won', "won't", 'would', 'wouldn', "wouldn't", 'y', 'yet', 'you',
       "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', '', ';', 'because', 'tv', "'s", '--', 
       'wo', 'some', '-', 'de', 'ca', 'so', "'ll", "'m", 'despite', 'two', 'should', 'might', "'d", 'inside', 'three', 'be', 'like',
        ')', '.', '...', '``', 'though', 'will', "'", 'each', "''", ',', 'since', 'every', '?', '(', ':', '`', 'us', 'go', '!', 'do',
        'I','So','"','|']

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model.cuda()
load_model_path = '/data/zhanghData/AttentionDefense/save_models/imdb_bert/base/best.pt'
model.load_state_dict(torch.load(load_model_path))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [10]:
model_type = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_type)
config = RobertaConfig.from_pretrained(model_type, num_labels=2, output_attentions=False, output_hidden_states=False, \
                attention_probs_dropout_prob=0, hidden_dropout_prob=0.1)
model = RobertaForSequenceClassification.from_pretrained(model_type, config=config) 
model.cuda()
load_model_path = '/home/zhangh/AttentionDefense/save_models/imdb_roberta/base/best.pt'
model.load_state_dict(torch.load(load_model_path))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

FileNotFoundError: [Errno 2] No such file or directory: '/home/zhangh/AttentionDefense/save_models/imdb_roberta/base/best.pt'

In [6]:
##攻击文件导入
from attack_instance import read_adv_files
atk_path = '/data/zhanghData/AttentionDefense/save_results/attacks/imdb_bert/base/pwws_0.1_100.csv'
atk_instances = read_adv_files(atk_path)
all_orig_texts = [atk.orig_text for atk in atk_instances]
all_perd_texts = [atk.perd_text for atk in atk_instances]
all_gold_labels = [atk.ground for atk in atk_instances]
all_changed_indices = [atk.atk_indices for atk in atk_instances]
all_change_words = [atk.atk_changes for atk in atk_instances] 